# Bac à sable RAG

Ce notebook part du principe que la _vector database_ est déjà prête, c'est-à-dire que les étapes suivantes ont déjà été faites:

<div>
<img src="https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png" width="500"/>
</div>

Nous nous intéressons à celles-ci:

<div>
<img src="https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png" width="500"/>
</div>


In [1]:
import logging
import os
import s3fs

from langchain.schema.runnable.config import RunnableConfig
from langchain_core.prompts import PromptTemplate

from src.chain_building.build_chain import build_chain
from src.chain_building.build_chain_validator import build_chain_validator
from src.db_building import (
    load_retriever,
    load_vector_database
)
from src.model_building import build_llm_model
from src.utils.formatting_utilities import add_sources_to_messages, str_to_bool



/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import de la database et du modèle génératif

### Base de données vectorielle

In [2]:
import os
from src.config import process_args, DefaultFullConfig, FullConfig, custom_config
from src.model_building import cache_model_from_hf_hub

EMB_MODEL_NAME = "OrdalieTech/Solon-embeddings-large-0.1"
LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"

hf_token = os.environ["HF_TOKEN"]
s3_token = os.environ["AWS_SESSION_TOKEN"]

cache_model_from_hf_hub(EMB_MODEL_NAME, hf_token=hf_token, s3_token=s3_token)
cache_model_from_hf_hub(LLM_MODEL, hf_token=hf_token, s3_token=s3_token)

DATABASE_RUN_ID = "9c9c411829c947799e3acd3df1564c0b"

# Create a custom confz configuration
config = custom_config(
    defaults = { # These defaults can be overriden with env variables
        "MAX_NEW_TOKENS": 2000,
        "MODEL_TEMPERATURE": 1.0,
        "quantization": True,
        "mlflow_run_id": DATABASE_RUN_ID,
    },
    overrides = { # These values are going to be used no matter what
        "UVICORN_TIMEOUT_KEEP_ALIVE": 0,
        "MAX_NEW_TOKENS": 2000,
        "LLM_MODEL": LLM_MODEL,
        "EMB_MODEL_NAME": EMB_MODEL_NAME,
        "mlflow_run_id": DATABASE_RUN_ID,
    })
RETURN_FULL_TEXT = True
DO_SAMPLE = True

CLI_MESSAGE_SEPARATOR = (config.cli_message_separator_length * "-") + " \n"

# Remote file configuration
fs = s3fs.S3FileSystem(endpoint_url=config.s3_endpoint_url)

Model OrdalieTech/Solon-embeddings-large-0.1 found in local cache. 
Model mistralai/Mistral-7B-Instruct-v0.3 found in local cache. 


In [3]:
db = load_vector_database(filesystem=fs, config=config)

In [4]:
#f"Nombre de documents dans la vector db: {len(db.get()['documents'])}"

## La chaine tout en un (avec langchain)

In [ ]:
from langchain_community.llms import VLLM

retriever, vectorstore = load_retriever(
        vectorstore=db,
        retriever_params={"search_type": "similarity", "search_kwargs": {"k": 10}},
    )

MODEL_TO_ARGS = {
    "mistralai/Mistral-7B-Instruct-v0.3": {
        "tokenizer_mode": "mistral",
        "config_format": "mistral",
        "load_format": "mistral",
        "enforce_eager": True,
    },
    "mistralai/Ministral-8B-Instruct-2410": {
        "tokenizer_mode": "mistral",
        "config_format": "mistral",
        "load_format": "mistral",
        "enforce_eager": True,
    },
    "mistralai/Mistral-Small-Instruct-2409": {
        "tokenizer_mode": "mistral",
        "config_format": "mistral",
        "load_format": "mistral",
        "enforce_eager": True,
    },
    "Qwen/Qwen2.5-1.5B-Instruct": {},
    "Qwen/Qwen2.5-32B-Instruct": {
        "max_model_len": 20000,
        "gpu_memory_utilization": 0.95,
        "enforce_eager": True,
    },
    "Qwen/Qwen2.5-72B-Instruct-GPTQ-Int4": {
        "max_model_len": 8192,
        "gpu_memory_utilization": 1.0,
        "enforce_eager": True,
        "enable_chunked_prefill": True,
        "max_num_batched_tokens": 2048,
    },
    "meta-llama/Meta-Llama-3.1-8B-Instruct": {},
    "hugging-quants/Meta-Llama-3.1-70B-Instruct-GPTQ-INT4": {
        "max_model_len": 8192,
        "gpu_memory_utilization": 1.0,
        "enforce_eager": True,
        "enable_chunked_prefill": True,
        "max_num_batched_tokens": 2048,
    },
    "neuralmagic/Llama-3.1-Nemotron-70B-Instruct-HF-FP8-dynamic": {
        "max_model_len": 8192,
        "gpu_memory_utilization": 1.0,
        "enforce_eager": True,
        "enable_chunked_prefill": True,
        "max_num_batched_tokens": 2048,
    },
}

llm = VLLM(model=LLM_MODEL, **MODEL_TO_ARGS.get(LLM_MODEL, {}))

2024-12-04 09:47:22,910	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 12-04 09:47:29 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 12-04 09:47:29 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='mistralai/Mistral-7B-Instruct-v0.3', speculative_config=None, tokenizer='mistralai/Mistral-7B-Instruct-v0.3', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=mistralai

/opt/conda/lib/python3.12/site-packages/vllm/transformers_utils/tokenizer_group/tokenizer_group.py:23: FutureWarning: It is strongly recommended to run mistral models with `--tokenizer_mode "mistral"` to ensure correct encoding and decoding.
  self.tokenizer = get_tokenizer(self.tokenizer_id, **tokenizer_config)


INFO 12-04 09:47:30 selector.py:135] Using Flash Attention backend.
INFO 12-04 09:47:30 model_runner.py:1072] Starting to load model mistralai/Mistral-7B-Instruct-v0.3...
INFO 12-04 09:47:30 weight_utils.py:243] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:00<00:01,  1.15it/s]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:01<00:00,  1.10it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.12it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.12it/s]



INFO 12-04 09:47:34 model_runner.py:1077] Loading model weights took 13.5083 GB
INFO 12-04 09:47:35 worker.py:232] Memory profiling results: total_gpu_memory=93.00GiB initial_memory_usage=16.20GiB peak_torch_memory=19.00GiB memory_usage_post_profile=16.29GiB non_torch_memory=0.67GiB kv_cache_size=64.03GiB gpu_memory_utilization=0.90
INFO 12-04 09:47:35 gpu_executor.py:113] # GPU blocks: 32784, # CPU blocks: 2048
INFO 12-04 09:47:35 gpu_executor.py:117] Maximum concurrency for 32768 tokens per request: 16.01x
INFO 12-04 09:47:37 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-04 09:47:37 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.


In [6]:
from src.utils import create_prompt_from_instructions, format_docs


system_instructions = """
Tu es un assistant spécialisé dans la statistique publique.
Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.

Réponds en FRANCAIS UNIQUEMENT. Utilise une mise en forme au format markdown.

En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.

La réponse doit être développée et citer ses sources (titre et url de la publication) qui sont référencées à la fin.
Cite notamment l'url d'origine de la publication, dans un format markdown.

Cite 5 sources maximum.

Tu n'es pas obligé d'utiliser les sources les moins pertinentes.

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.

Voici le contexte sur lequel tu dois baser ta réponse :
Contexte: {context}
"""

question_instructions = """
Voici la question à laquelle tu dois répondre :
Question: {question}

Réponse:
"""

prompt = create_prompt_from_instructions(system_instructions, question_instructions)


In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
    )

In [10]:
answer_pib = rag_chain.invoke("Quelle est la définition du PIB ?")

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.56s/it, est. speed input: 612.17 toks/s, output: 137.17 toks/s]


In [9]:
answer_pib

" Le PIB (Produit Intérieur Brut) représente la somme des valeurs marchandes des produits et des services, qu'ils soient encore en phase de production ou non, qui sont produits et consommés dans un territoire donné durant une période donnée. C'est donc la quantification de l'activité économique du pays, globalement monétaire, évaluée au cours d'un certain temps.\n\n     La source de cette définition se trouve dans la [présentation des définitions sur le site internet de l'Insee](https://www.insee.fr/fr/statistiques/4162165). Plus précisément, elle est décrite lors de la présentation du bilan économique pour l'année 2018.\n\n     Il est à noter que la définition du PIB varie énormément selon les contextes. L'Insee présente des détails supplémentaires dans uneINTRODUCTION GAOM PAR ÎTÉLÉPHÉRIE RAGADEAU, [AUTRE SOURCE](https://www.insee.fr/fr/statistiques/4258088). En effet, la source fournit une introduction détaillée sur la théorie économique du PIB et son histoire.\n\n     Pour en savoi

In [11]:
from IPython.display import display, Markdown, Latex

display(
    Markdown(
        answer_pib.replace("   ", "")
    )
)

 Le PIB (Produit intérieur brut) est un indicateur économique important qui mesure l'ensemble des biens et des services produits par une économie en un fonctionnement normal sur un laps de temps donné. Il est calculé, à l'échelle nationale, sur la base des propriétés des entreprises, appels de mine, fermes, partenaires du secteur non marchand, statistiques harmonisées de la production de petites entreprises et de la consommation de ménages, régionaux et fins de compte. La (source pour la définition du PIB) donne plus de details sur cette définition économique. En outre, il est possible de trouver des exemples à partir de la définition de la publication (Doc 1 et Doc 5) sur le bilan économique de l'année 2016, 2017, 2018.

In [10]:
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [11]:
for chunk in rag_chain_with_source.stream("Quelle est la définition du PIB ?"):
    print(chunk)

{'question': 'Quelle est la définition du PIB ?'}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 100.25it/s]


{'context': [Document(metadata={'Header 1': 'Définitions', 'Header 2': 'Bilan économique 2018', 'categorie': 'Publications grand public', 'collection': 'Bilans économiques', 'dateDiffusion': '2019-06-06 10:00', 'id': '', 'libelleAffichageGeo': 'Bretagne', 'sousTitre': '', 'theme': 'Économie générale (inflation, PIB, dette,...)', 'titre': 'Définitions', 'url': 'https://www.insee.fr/fr/statistiques/4162165', 'xml_auteurs': '', 'xml_content': '', 'xml_intertitre': ''}, page_content='# Définitions  \n## Bilan économique 2018'), Document(metadata={'Header 1': 'Définitions', 'Header 2': 'Bilan économique 2016', 'categorie': 'Publications grand public', 'collection': 'Bilans économiques', 'dateDiffusion': '2017-05-23 10:00', 'id': '', 'libelleAffichageGeo': 'Occitanie', 'sousTitre': '', 'theme': 'Conjoncture', 'titre': 'Définitions', 'url': 'https://www.insee.fr/fr/statistiques/2852600', 'xml_auteurs': '', 'xml_content': '', 'xml_intertitre': ''}, page_content='# Définitions  \n## Bilan écono

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.69s/it, est. speed input: 259.01 toks/s, output: 138.86 toks/s]

{'answer': ' Le PIB (Produit intérieur brut) est la somme des valeurs des biens et services manufacturés, immateriaux, agricoles et de services fournis au cours d’une année par les différents secteurs productifs d’une économie[1](https://www.insee.fr/fr/statistiques/4258088).\n\n     Plus précisément, le PIB est un indicateur statistique qui mesure la valeur totale des activités productives d’une économie pendant une année donnée en termes de revenu, dans le sens où il représente l’ensemble des biens et services que les secteurs productifs ont pu produire pendant cette année[2](https://www.insee.fr/fr/statistiques/4162165).\n\n     Il est calculé pour chaque année et nous disposons des données pour les années 2016 [3](https://www.insee.fr/fr/statistiques/2852600), 2017 [4](https://www.insee.fr/fr/statistiques/3548949), ainsi que pour la plus récente année 2018 [5](https://www.insee.fr/fr/statistiques/4162165).\n\n     Les données intéressantes pour le PIB sont aussi la part des dépense

In [15]:
retriever.invoke("Quelle est la définition du PIB ?")[:5]

Batches: 100%|██████████| 1/1 [00:00<00:00, 71.34it/s]


[Document(metadata={'Header 1': 'Définitions', 'Header 2': 'Bilan économique 2018', 'categorie': 'Publications grand public', 'collection': 'Bilans économiques', 'dateDiffusion': '2019-06-06 10:00', 'id': '', 'libelleAffichageGeo': 'Bretagne', 'sousTitre': '', 'theme': 'Économie générale (inflation, PIB, dette,...)', 'titre': 'Définitions', 'url': 'https://www.insee.fr/fr/statistiques/4162165', 'xml_auteurs': '', 'xml_content': '', 'xml_intertitre': ''}, page_content='# Définitions  \n## Bilan économique 2018'),
 Document(metadata={'Header 1': 'Définitions', 'Header 2': 'Bilan économique 2016', 'categorie': 'Publications grand public', 'collection': 'Bilans économiques', 'dateDiffusion': '2017-05-23 10:00', 'id': '', 'libelleAffichageGeo': 'Occitanie', 'sousTitre': '', 'theme': 'Conjoncture', 'titre': 'Définitions', 'url': 'https://www.insee.fr/fr/statistiques/2852600', 'xml_auteurs': '', 'xml_content': '', 'xml_intertitre': ''}, page_content='# Définitions  \n## Bilan économique 2016'

## La chaine décomposée

### Modèle génératif

In [6]:
import os
from langchain_community.llms import VLLM

MAX_NEW_TOKEN = 8192
TEMPERATURE = 0.2
REP_PENALTY = 1.1
TOP_P = 0.8

hf_token = os.environ["HF_TOKEN"]
s3_token = os.environ["AWS_SESSION_TOKEN"]

#cache_model_from_hf_hub(EMB_MODEL_NAME, hf_token=hf_token, s3_token=s3_token)
#cache_model_from_hf_hub(LLM_MODEL, hf_token=hf_token, s3_token=s3_token)
LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"

llm = VLLM(
        model=LLM_MODEL,
        max_new_tokens=MAX_NEW_TOKEN,
        top_p=TOP_P,
        temperature=TEMPERATURE,
        rep_penalty=REP_PENALTY,
        tokenizer_mode = "mistral",
        config_format =  "mistral",
        load_format = "mistral",
        enforce_eager = True
        )

2024-12-03 15:28:54,342	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 12-03 15:29:02 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 12-03 15:29:02 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='mistralai/Mistral-7B-Instruct-v0.3', speculative_config=None, tokenizer='mistralai/Mistral-7B-Instruct-v0.3', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=mistralai

/opt/conda/lib/python3.12/site-packages/vllm/transformers_utils/tokenizer_group/tokenizer_group.py:23: FutureWarning: It is strongly recommended to run mistral models with `--tokenizer_mode "mistral"` to ensure correct encoding and decoding.
  self.tokenizer = get_tokenizer(self.tokenizer_id, **tokenizer_config)


INFO 12-03 15:29:02 selector.py:135] Using Flash Attention backend.
INFO 12-03 15:29:04 model_runner.py:1072] Starting to load model mistralai/Mistral-7B-Instruct-v0.3...
INFO 12-03 15:29:04 weight_utils.py:243] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:00<00:01,  1.19it/s]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:01<00:00,  1.15it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.15it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.15it/s]



INFO 12-03 15:29:52 model_runner.py:1077] Loading model weights took 13.5083 GB
INFO 12-03 15:29:54 worker.py:232] Memory profiling results: total_gpu_memory=93.00GiB initial_memory_usage=14.12GiB peak_torch_memory=16.92GiB memory_usage_post_profile=14.22GiB non_torch_memory=0.68GiB kv_cache_size=66.11GiB gpu_memory_utilization=0.90
INFO 12-03 15:29:54 gpu_executor.py:113] # GPU blocks: 33847, # CPU blocks: 2048
INFO 12-03 15:29:54 gpu_executor.py:117] Maximum concurrency for 32768 tokens per request: 16.53x
INFO 12-03 15:29:56 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-03 15:29:56 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.


In [12]:
llm.generate(["La recette de la tarte tatin", "tu fais qupoi ?", "où va le monde"])

Processed prompts: 100%|██████████| 3/3 [00:03<00:00,  1.18s/it, est. speed input: 6.21 toks/s, output: 301.60 toks/s]


LLMResult(generations=[[Generation(text='\n\nIngredients:\n\n- 100g of sugar\n- 100g of butter\n- 4 apples\n- 1 sheet of puff pastry\n- 1 pinch of salt\n\nInstructions:\n\n1. Preheat the oven to 200°C.\n2. In a pan, melt the sugar and butter over medium heat until the sugar has dissolved.\n3. Arrange the apples in the pan, cut side down, and cook for about 10 minutes or until the apples are golden brown.\n4. Roll out the puff pastry on a floured surface and cut out a circle that is slightly larger than the diameter of the pan.\n5. Place the pastry over the apples and tuck the edges in around the apples.\n6. Bake in the oven for about 20 minutes or until the pastry is golden brown.\n7. Let the tarte cool for a few minutes before inverting it onto a plate.\n8. Serve with whipped cream or vanilla ice cream.\n\nEnjoy your delicious tarte tatin!')], [Generation(text='\n\n# What is the difference between a "bouquet" and a "bouquet garni"?\n\nA "bouquet" is a mixture of herbs, spices, and som

## Transform vector database into retriever

In [ ]:
retriever, vectorstore = load_retriever(
                emb_model_name=embedding,
                persist_directory=CHROMA_DB_LOCAL_DIRECTORY,
                vectorstore=db,
                retriever_params={
                    "search_type": "similarity",
                    "search_kwargs": {"k": 30}
                },
            )

2024-11-20 02:21:02 PM vectorstore being provided, skipping the reloading


In [ ]:
retriever.invoke("Chiffres du chômage")[:5]

Batches: 100%|██████████| 1/1 [00:00<00:00, 103.74it/s]


[Document(metadata={'Header 1': 'Chômage et halo autour du chômage en 2019', 'Header 2': 'Résumé :', 'Header 3': "Téléchargement des tableaux à l'unité", 'Header 4': 'Chômage', 'categorie': 'Chiffres détaillés', 'collection': '', 'dateDiffusion': '2020-06-23 12:00', 'libelleAffichageGeo': 'France', 'theme': 'Emploi – Population active', 'titre': 'Chômage et halo autour du chômage en 2019', 'url': 'https://www.insee.fr/fr/statistiques/4498582'}, page_content='#### Chômage'),
 Document(metadata={'Header 1': 'Emploi-Chômage', 'categorie': 'Publications pour expert', 'collection': 'Note de conjoncture', 'dateDiffusion': '2020-12-15 17:00', 'libelleAffichageGeo': 'France', 'theme': 'Économie générale (inflation, PIB, dette,...)', 'titre': 'Emploi-Chômage', 'url': 'https://www.insee.fr/fr/statistiques/4653872'}, page_content='# Emploi-Chômage'),
 Document(metadata={'Header 1': 'Chômage', 'categorie': 'Publications pour expert', 'collection': 'Note de conjoncture', 'dateDiffusion': '2017-03-1

In [19]:
retriever.batch(["Quelle est la définition du PIB ?", "Où sont les chiffres de l'inflation ?"])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 31.03it/s]


[[Document(metadata={'Header 1': 'Innover pour résister à la crise ou se développer à l’export', 'Header 2': 'Définitions :', 'categorie': 'Publications grand public', 'collection': 'Insee Première', 'dateDiffusion': '2012-10-23 22:00', 'id': '', 'libelleAffichageGeo': 'France', 'sousTitre': '', 'theme': 'Mondialisation, compétitivité et innovation', 'titre': 'Innover pour résister à la crise ou se développer à l’export', 'url': 'https://www.insee.fr/fr/statistiques/1280888', 'xml_auteurs': '', 'xml_content': '', 'xml_intertitre': ''}, page_content='## Définitions :'),
  Document(metadata={'Header 1': 'Les comptes des administrations publiques en 2011', 'Header 2': 'Résumé :', 'Header 3': 'Le déficit des administrations publiques centrales continue de se réduire', 'categorie': 'Publications grand public', 'collection': 'Insee Première', 'dateDiffusion': '2012-05-15 05:30', 'id': '', 'libelleAffichageGeo': 'France', 'sousTitre': '', 'theme': 'Finances publiques', 'titre': 'Les comptes d

## RAG mode: on 

In [34]:
from utils import (
    format_docs,
    create_prompt_from_instructions,
    retrieve_db_from_cache,
)

system_instructions = """
Tu es un assistant spécialisé dans la statistique publique. Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.

Réponds en FRANCAIS UNIQUEMENT. Utilise une mise en forme au format markdown.

En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.

La réponse doit être développée et citer ses sources (titre et url de la publication) qui sont référencées à la fin. Cite notamment l'url d'origine de la publication, dans un format markdown.

Cite 5 sources maximum.

Tu n'es pas obligé d'utiliser les sources les moins pertinentes. 

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.

Voici le contexte sur lequel tu dois baser ta réponse :
Contexte: {context}
"""

question_instructions = """
Voici la question à laquelle tu dois répondre :
Question: {question}

Réponse:
"""


prompt = create_prompt_from_instructions(system_instructions, question_instructions)


In [37]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

## Imbrication dans langchain
rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

In [38]:
rag_chain.invoke("Chiffres du chômage")

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.11s/it, est. speed input: 675.99 toks/s, output: 78.27 toks/s]


"1. Le chômage en France est mesuré par le taux de chômage, qui est le rapport entre le nombre de chômeurs au sens du BIT et le nombre de personnes en emploi ou au chômage. Le BIT (Bénéficiaire d'allocations chômage) est une personne qui reçoit des allocations chômage, c'est-à-dire des allocations de l'État pour couvrir ses frais de subsistance pendant qu'elle est au chômage.\n\n    2. Le taux de chômage en France a été stable depuis plusieurs années, mais il varie selon les régions et les catégories socioprofessionnelles. Par exemple, en 2019, le taux de chômage en France métropolitaine était de 7,7%, mais il était plus élevé en région Île-de-France (9,3%) et plus bas en région Centre-Val de Loire (6,1%).\n\n    3. Les chômeurs en France sont majoritairement des hommes (54,6% en 2019), mais leur proportion est en baisse depuis plusieurs années. Les chômeurs sont également majoritairement des personnes de 30 à 49 ans (68,1% en 2019), mais leur proportion est en hausse depuis plusieurs 

In [40]:
for chunk in rag_chain.stream("A quelle échelle puis-je trouver le taux de chîmage ? "):
    print(chunk, end="", flush=True)

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.83s/it, est. speed input: 1743.61 toks/s, output: 66.77 toks/s]

 Le taux de chômage peut être trouvé à l'échelle de la région, du département ou de la commune. Il est calculé en divisant le nombre de personnes chômeures par le nombre de personnes actives de la population.

     Pour trouver le taux de chômage à l'échelle de la région, il est nécessaire de consulter les données de l'Insee sur le taux de chômage régional. Par exemple, pour la région Grand Est, vous pouvez consulter la publication "Taux de chômage régional" sur le site de l'Insee : https://www.insee.fr/fr/statistiques/7931214

     Pour trouver le taux de chômage à l'échelle du département, il est nécessaire de consulter les données de l'Insee sur le taux de chômage départemental. Par exemple, pour le département de la Nièvre, vous pouvez consulter la publication "Taux de chômage départemental" sur le site de l'Insee : https://www.insee.fr/fr/statistiques/5370760

     Pour trouver le taux de chômage à l'échelle de la commune, il est nécessaire de consulter les données de l'Insee sur 